In [17]:
# the following code allows you to import function from
# jupyter notebook in other directory
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [18]:
import functools
%run ../gw/gw_wrapped_compressed.ipynb
%run ../compressed_class_list/compressed_class_list.ipynb

In [19]:
@functools.lru_cache(maxsize=None)
def chiang_ogw_invariant(n, b, k, A:ClassList):
    l = len(A)

    if A.invalid_class_list:
        return 0

    # wall-crossing theorem (Elad)
    if l > 0 and A[0] == -1:
        return -chiang_ogw_invariant(n, b, k+1, A[:-1])

    #degree axiom (Elad)
    if 2*sum(A) != (n - 3 + 2*b + k + 2*l - k*n):  
        # The maslov index is twice the chern class n+1 for holomorphic spheres. 
        # Using the invoulotion we can deduce that its half of this for disks thus mu(b)=b*(n+1) 
        # p 595 mcduff 
        #ogw_d[str((n, b, k, A)) + ", degree axiom"] = 0
        return 0

    
    # other
    if k == 0 and is_even(b):
        return 0

    # zero axiom
    if b == 0:
        if k == 1 and l == 1 and A[0] == 0:
            return -1
        else:
            return 0

    # unit axiom (May)
    # handle all beta = 0 cases
    if l > 0 and A[0] == 0:
        if b == 0 and k == 1 and l == 1:
            return -1
        else:
            return 0

    if b == 1 and k == 1 and A == []:
        return 3

    if l >= 2:
        cor = ogw_invariant(n, b, k, [A[0]-1, A[1]+1] + A[2:])
        col = 0
        ool = 0
        oor = 0
        A_0 = A[0]
        A_1 = A[1]

        for t in A[2:].bipart():
            I1 = t[0]
            I2 = t[1]
            bipart_coeff = ClassList.bipart_coeff(A[2:], I1)

            for i in range(0, n+1):

                L1_cor = [1, A_1, i] + I1
                L2_cor = [n-i, A_0-1] + I2 
                for b1 in range(1, b):
                    b_ = 0
                    if (b - b1) % 2 == 0:
                        b_ = (b - b1) / 2
                    else:
                        continue
                    
                    cor += bipart_coeff \
                        * (-1)^(b_*(n+1)/2) *  gw_invariant(n, b_, L1_cor) \
                        *  chiang_ogw_invariant(n, b1, k, L2_cor) 

                        # I1 = [1,1,1,1,2,2,2]
                        # I1.c_list = [0, 4, 3, 0]
                        # [1, 2, 3] + I1
                        # I1 + [1, 2, 3] = [0, 5, 4, 1]

            for i in range(0, n+1):

                L1_col = [1, A[0] - 1, i] + I1 
                L2_col = [n-i, A[1]] + I2 
                for b1 in range(1, b):
                    b_ = 0
                    if (b - b1) % 2 == 0:
                        b_ = (b - b1) / 2
                    else:
                        continue

                    col += bipart_coeff \
                        * (-1)^((n+1)*b_/2)* gw_invariant(n, b_, L1_col) \
                        * chiang_ogw_invariant(n, b1, k, L2_col)


            L1_ool = [1, A[0]-1] + I1
            L2_ool = [A[1]] + I2
            for b1 in range(1, b):
                b2 = b - b1

                for k1 in range(0, k+1):
                    k2 = k - k1
                    ool += bipart_coeff \
                        * binomial(k, k1)*chiang_ogw_invariant(n, b1, k1, L1_ool) \
                        * chiang_ogw_invariant(n, b2, k2 + 1, L2_ool)


            L1_oor = [1, A[1]] + I1
            L2_oor = [A[0] - 1] + I2    
            for b1 in range(1, b):
                b2 = b - b1

                for k1 in range(0, k+1):
                    k2 = k - k1

                    oor += bipart_coeff \
                        * binomial(k, k1) * chiang_ogw_invariant(n, b1, k1, L1_oor) \
                        * chiang_ogw_invariant(n, b2, k2 + 1, L2_oor)

        return cor - col - oor + ool

    if l >= 1:
        co = 0
        ool = 0
        oor = 0
        A_0 = A[0]

        for t in A[2:].bipart():
            I1 = t[0]
            I2 = t[1]

            bipart_coeff = ClassList.bipart_coeff(A[2:], I1)
            
            for i in range(0, n+1):
                L1_co = [A_0 - 1, 1, i] + I1
                L2_co = [n - i] + I2

                for b1 in range(1, b):
                    b_ = 0
                    if (b - b1) % 4 == 0:
                        b_ = (b - b1) / 4
                    else:
                        continue

                    co += bipart_coeff \
                        * gw_invariant(n, b_, L1_co) \
                        * chiang_ogw_invariant(n, b, k, L2_co)

            L1_ool = [A_0 - 1] + I1
            L2_ool = I2
            for b1 in range(0, b+1):
                b2 = b - b1

                for k1 in range(0, k-1):
                    k2 = k - 1 - k1
                    ool += bipart_coeff \
                        * binomial(k-1, k1) \
                        * chiang_ogw_invariant(n, b1, k1, L1_ool) \
                        * chiang_ogw_invariant(n, b2, k2 + 2, L2_ool)
                    

            L1_oor = [A_0 - 1] + I1
            L2_oor = [1] + I2
            for b1 in range(0, b+1):
                b2 = b - b2

                for k in range(0, k-1):
                    k2 = k - 1 - k1
                    oor += bipart_coeff \
                        * binomial(k-1, k1) \
                        * chiang_ogw_invariant(n, b1, k1 + 1, L1_oor) \
                        * chiang_ogw_invariant(n, b2, k2 + 1, L2_oor)

            return -co + ool - oor

In [20]:
print(chiang_ogw_invariant(3, 1, 0, ClassList([0, 0, 1, 0])))

0
